In [1]:
import pkuseg
import pandas as pd

data = pd.read_csv('../data/gwt_info.csv')

In [2]:
def getStopwords(path):
    '''
    读取停用词列表
    '''
    with open(path, 'r', encoding='utf-8') as f:
        stopword_list = [word.strip('\n') for word in f.readlines()]
    return stopword_list

stopwords = getStopwords('../data/cn_stopwords.txt')

In [3]:
seg = pkuseg.pkuseg()
def clean_cn_str(str,stopword_list):
    '''
    为中文字符串分词后去停用词，返回中文分词列表
    '''
    result = []
    word_list = seg.cut(str.strip('\n'))
    for word in word_list:
        if word not in stopword_list:
            result.append(word)
    return result


In [4]:
data['words'] = data.apply(lambda x: clean_cn_str(str(x['title'])+' '+str(x['content']),stopwords),axis=1)

In [9]:
data['words']

0       [做好, 我校, 2021-2022, 学年, 第一, 学期, 期末, 假期, 疫情, 防控...
1       [召开, 党委, 常委会, （, 扩大, ）, 会议, 暨, 基层, 党委, （, 总支, ...
2       [参加, 2022年, 新年, 音乐会, 通知, 丰富, 校园, 文化, 生活, 学校, 定...
3       [召开, 校, 党委, 班子, 成员, 述责, 述廉, 测评, 会议, 暨, 中层, 干部,...
4       [参加, 深圳, 大学, 2021年, 奖学金, 颁奖, 典礼, 通知, 全, 校, 单位,...
                              ...                        
1395    [面向, 循环, 经济, 碳, 纤维, 增强, 复合, 材料, 回收, 制造, 项目, 拨付...
1396    [深圳, 大学, 龙岗, 创新, 研究院, 2020, 年度, 总结, 暨, 表彰, 大会,...
1397    [三维, 重建, 前沿, 技术, 研究, 项目, 拨付, 测试, 化验, 加工, 经费, 公...
1398    [征集, 2021, 年度, 中, 韩, 青年, 科学家, 交流, 计划, 中国, 青年, ...
1399    [申报, 2021, 年度, 腾讯, 科学, 探索, 奖, 通知, 科学, 探索, 奖, 面...
Name: words, Length: 1400, dtype: object

In [10]:
data[['department','words']].to_csv('../data/gwt_corpus.csv')

In [11]:
# 取出各类最近30条，作为特征选择的语料
docs = data[data['no']<30][['department','words']]
docs

,department,words
0,党政办公室,"[做好, 我校, 2021-2022, 学年, 第一, 学期, 期末, 假期, 疫情, 防控..."
1,党政办公室,"[召开, 党委, 常委会, （, 扩大, ）, 会议, 暨, 基层, 党委, （, 总支, ..."
2,党政办公室,"[参加, 2022年, 新年, 音乐会, 通知, 丰富, 校园, 文化, 生活, 学校, 定..."
3,党政办公室,"[召开, 校, 党委, 班子, 成员, 述责, 述廉, 测评, 会议, 暨, 中层, 干部,..."
4,党政办公室,"[参加, 深圳, 大学, 2021年, 奖学金, 颁奖, 典礼, 通知, 全, 校, 单位,..."
...,...,...
851,科学技术部,"[转发, 深圳市, 科技, 创新, 委员会, 国家, 广东省, 科技, 奖, 配套, 奖励,..."
852,科学技术部,"[申请, 2021, 年度, 国家, 自然科学, 基金, 应急, 管理, 项目, 全面, 推..."
853,科学技术部,"[开展, 深圳, 大学, 2021, 年度, 科研, 经费, 使用, 管理, 专项, 自查,..."
854,科学技术部,"[申请, 2022, 年度, 国家, 自然科学, 基金, 委员会, 加拿大, 卫生, 研究院..."


## 1. 互信息

统计词表中的词在每类文档的出现次数

In [15]:
vocabulary = {}
_class = docs['department'].unique()
cnt_class = {}
for c in _class:
    cnt_class[c] = 0
cnt_all = 0
for index,row in docs.iterrows():
    words = list(set(row['words']))
    for word in words:
        if word not in vocabulary:
            vocabulary[word] = {}
            for c in _class:
                vocabulary[word][c] = 0
        vocabulary[word][row['department']] = vocabulary[word][row['department']]+1
        cnt_class[row['department']] = cnt_class[row['department']]+1
        cnt_all += 1
        

In [22]:
import math

MI = {}
for c in _class:
    MI[c] = []
    
for word,word_class_cnt in vocabulary.items():
    for c,cnt in word_class_cnt.items():
        n = cnt_all
        n11 = cnt
        n01 = cnt_class[c] - n11 
        n1_ = sum(word_class_cnt.values())
        n10 = n1_ - n11
        n00 = n - n1_ - n01
        n0_ = n - n1_
        n_1 = n01 + n11
        n_0 = n - n1_
        mi = (n11/n)*math.log((n*n11+1)/n1_/n_1) + (n01/n)*math.log((n*n01+1)/n0_/n_1) + (n10/n)*math.log((n*n10+1)/n1_/n_0) + (n00/n)*math.log((n*n00+1)/n0_/n_0)
        MI[c].append((word,mi))

for c in MI.values():
    c.sort(key=lambda x:x[1],reverse=True)
    
for cname,ccnt in MI.items():
    print(cname,' Top 15: ',ccnt[:15])
    print('\n')

党政办公室  Top 15:  [('2021年', -0.14197210966412985), ('）', -0.1427317401633203), ('（', -0.1427317401633203), ('工作', -0.14287912795378307), ('单位', -0.14316455679153353), ('通知', -0.14321257854379363), ('大学', -0.14351543014442605), ('请', -0.14363647752703232), ('12月', -0.14364508507683252), ('深圳', -0.1436714392196878), ('学校', -0.14375497447213217), ('要求', -0.1438313295089455), ('学院', -0.1439926452992955), ('中', -0.14401825235809768), ('办公室', -0.14403081004716917)]


教务部  Top 15:  [('2021年', -0.1652252142841059), ('）', -0.1659793450554689), ('（', -0.1659793450554689), ('工作', -0.16607529177624458), ('通知', -0.1663870418363987), ('单位', -0.16642990571993302), ('大学', -0.1666630968282318), ('12月', -0.16671694842332904), ('深圳', -0.16683261002036695), ('请', -0.1668461716577026), ('学校', -0.16692412218914715), ('要求', -0.16701768158697244), ('学院', -0.167133541515803), ('进行', -0.16717021369774843), ('中', -0.16717210960381615)]


招生办公室  Top 15:  [('2021年', -0.183697524753985), ('）', -0.18419820716848354),

In [23]:
vocabulary['大学']

{'党政办公室': 16, '教务部': 21, '招生办公室': 25, '研究生院': 14, '科学技术部': 17}

In [24]:
len(vocabulary)

6728

In [25]:
cnt_class

{'党政办公室': 4219, '教务部': 4971, '招生办公室': 5594, '研究生院': 4860, '科学技术部': 6719}

n01 = cnt_class[c] - n11 

n01 应该是 不包含t但属于类别c的【文档】数，每类文档数都为30，所以这里应该是30-n11

cnt_class[c] 统计的是在c类中出现过的词的数量

n = cnt_all

cnt_all 统计的是文档中出现过的词的数量，也就是cnt_class中value之和

而n应该为数据集中所有文档的数量，也就是30*5

### 修正

In [26]:
vocabulary = {}
_class = docs['department'].unique()
doccnt_class = {}
for c in _class:
    doccnt_class[c] = 0
doccnt_all = 0
for index,row in docs.iterrows():
    words = list(set(row['words']))
    for word in words:
        if word not in vocabulary:
            vocabulary[word] = {}
            for c in _class:
                vocabulary[word][c] = 0 # word 在 c 类文档中出现的文档数
        vocabulary[word][row['department']] = vocabulary[word][row['department']]+1
    doccnt_class[row['department']] = doccnt_class[row['department']]+1
    doccnt_all += 1 # iterrows遍历的每一行是一个文档

In [31]:
MI = {}
for c in _class:
    MI[c] = []
    
for word,word_class_doccnt in vocabulary.items():
    for c,doccnt in word_class_doccnt.items():
        n = doccnt_all
        n11 = doccnt    # word 在 c 类文档中出现的文档数，可能为0
        n01 = doccnt_class[c] - n11 # word 在 c 类文档中不出现的文档数, 即 c 类文档总数 - n11，可能为0
        n1_ = sum(word_class_doccnt.values()) # word 在所有文档中出现的文档数，不可能为0
        n10 = n1_ - n11 # word 在其他类文档中出现的文档数，即包含word但不属于 c 类的文档数，可能为0
        n00 = n - n1_ - n01 # 不包含word的其他类文档数，可能为0
        n0_ = n - n1_ # 所有文档（包含c类）中不包含word的文档数，可能为0
        n_1 = doccnt_class[c] # c 类文档总数，此例中不可能为0
        n_0 = doccnt_all - n_1 # 其他类文档总数，不可能为0
        mi = (n11/n)*math.log((n*n11+1)/(n1_*n_1+1)) + (n01/n)*math.log((n*n01+1)/(n0_*n_1+1)) + (n10/n)*math.log((n*n10+1)/(n1_*n_0+1)) + (n00/n)*math.log((n*n00+1)/(n0_*n_0+1))
        # 分子分母+1做平滑处理
        MI[c].append((word,mi))

for c in MI.values():
    c.sort(key=lambda x:x[1],reverse=True)
    
for cname,ccnt in MI.items():
    print(cname,' Top 15: ',ccnt[:15])
    print('\n')

党政办公室  Top 15:  [('党政', 0.20752815630975247), ('定于', 0.09541537016935553), ('培养', 0.07935822464670121), ('全', 0.07088623000338307), ('总支', 0.06769202935778154), ('办公室', 0.06297422453426305), ('南山区', 0.055860030585275805), ('筹', 0.055860030585275805), ('博物馆', 0.055860030585275805), ('申请', 0.054387447882864544), ('基层', 0.04984115230131111), ('本页', 0.04984115230131111), ('入口', 0.04984115230131111), ('10月', 0.0498334312265465), ('建议', 0.04860653228466512)]


教务部  Top 15:  [('教务部', 0.3060839992840807), ('教学', 0.12228977888277089), ('课程', 0.07837799249604296), ('课堂', 0.06769202935778154), ('任课', 0.060899559594558596), ('办公室', 0.05751573056407797), ('招生', 0.05592593514852144), ('研究生院', 0.054387447882864544), ('11月', 0.053934989461036996), ('本科', 0.053605863684116524), ('门', 0.04984115230131111), ('科研', 0.04671635468213304), ('教师', 0.046630246497562436), ('229', 0.04424886378799753), ('学时', 0.04424886378799753)]


招生办公室  Top 15:  [('中学', 0.3394371389893528), ('招生', 0.3093593267003605), ('进', 0